In [1]:
## Run when initialise the code
from Code.general_functions import *
import geopandas as gpd
import osmnx as ox
from geopandas import GeoDataFrame, GeoSeries
from osmnx import io

from pandas import DataFrame

project_crs = 'epsg:3857'
from sklearn.cluster import DBSCAN
from shapely.geometry import Point, LineString, MultiPolygon
import math
import warnings
import pandas as pd

from tqdm import tqdm
import ast
warnings.filterwarnings(action='ignore')
from momepy import extend_lines
import os
pjr_loc = os.path.dirname(os.getcwd())
from itertools import combinations
import numpy as np
from math import log2

In [2]:
# Preprocessing code
# Define the data folder path
# Construct the folder path for the data using the current working directory
data_folder = os.path.join(os.path.dirname(os.getcwd()), 'places', 'run_SB', 'Santa_barbara_California')

# Load the OSM data into a GeoDataFrame, specifying the 'edges' layer
my_gdf = gpd.read_file(os.path.join(data_folder, 'osm_data.gpkg'), layer='edges')

# Filter out unwanted road types (e.g., motorway, trunk, and their links)
to_remove = my_gdf[~my_gdf['highway'].isin(['motorway', 'trunk', 'motorway_link', 'trunk_link'])]

# Exclude rows where 'name' is NaN or an empty string, then convert CRS to project_crs
df_pro = to_remove[~(to_remove['name'].isna() | (to_remove['name'] == ''))].to_crs(project_crs)

# Function to safely convert valid string representations of lists to actual lists
def convert_to_list(s):
    """
    Converts a string representation of a list into a Python list.
    
    Parameters:
    - s (str): The string to convert.
    
    Returns:
    - The first element of the list if the string is a valid list representation.
    - The original value if conversion fails.
    """
    try:
        return ast.literal_eval(s)[0]  # Extract the first element if the string is a valid list
    except (ValueError, SyntaxError, TypeError):
        return s  # Return the original value if the conversion fails

# Apply the conversion function to 'name' and 'highway' columns
df_pro['name'] = df_pro['name'].apply(convert_to_list)
df_pro['highway'] = df_pro['highway'].apply(convert_to_list)


In [ ]:
## Spatial Join for Streets

# Variables for street name fields
str_name = 'name_left'  # Field representing the name of the current street
con_str_name = 'name_right'  # Field representing the name of the connected street

# Step 1: Copy the original network dataframe
# This ensures the original data remains unchanged during processing
df_analysis = df_pro.copy()

# Step 2: Perform a spatial join
# This operation joins the street data with itself to find connections
gpd.sjoin(df_analysis, df_pro)
s_join_analysis = gpd.sjoin(df_analysis, df_pro)

# Step 3: Filter streets to find only those with different names
# Filtering ensures we focus only on streets that are not self-connected
s_join_analysis2 = s_join_analysis[s_join_analysis[str_name] != s_join_analysis[con_str_name]]

# Step 4: Group the data by street name
# This prepares the data for analyzing connections
group_name = s_join_analysis2.groupby(str_name)

# Initialize a dictionary to store connected streets
dic_data = {}

def connected_streets(str_lines):
    """
    Populate the dictionary with the name of the current street and all its connected streets.

    :param str_lines: DataFrame containing street line information for a specific group
    :return: None
    """
    dic_data[str_lines[str_name].iloc[0]] = list(str_lines[con_str_name].unique())

# Step 5: Apply the function to group data to populate the dictionary
group_name.apply(connected_streets)

# Step 6: Save the data as a DataFrame
# Convert the dictionary to a DataFrame for easier analysis and storage
connections = pd.DataFrame(dic_data.items(), columns=['street_name', 'connections'])


In [16]:
# Spatial join for roundabout
roundabout =to_remove[((to_remove['junction'] == 'roundabout') | (to_remove['junction'] == 'circular'))]
# create polygon around each polygon and union
roundabout_buffer = roundabout['geometry'].buffer(cap_style=2,distance=10).unary_union
# Create a GeoDataFrame
roundabout_geo = gpd.GeoDataFrame({'geometry': roundabout_buffer.geoms}, crs="EPSG:4326")
s_join_roundabout = gpd.sjoin(df_pro,roundabout_geo)
streets_with_roundabout = s_join_roundabout.groupby('name')['index_right'].apply(lambda x: list(set(x)))

connections2= connections.set_index('street_name')
connections2['roundabout_id'] = streets_with_roundabout

In [34]:

connections2.to_csv(f'{pjr_loc}/test/connections.csv')

In [33]:
pjr_loc